# Gender Only UTMB Grade Adjusted Average Pace Predictions

This notebook predicts average pace using grade adjusted pace times, and runners similar to the query that are of equal gender. 

In [156]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

I aim to produce a challenging but not impossible average pace prediction for each runner. I will first get the runner's non personal best average pace and find the most similar non personal best average pace to that runner. I will then find the personal best average pace of these runners. By average these pb times, or getting the best personal best average pace of these runners, I will be able to produce a predicted average pace. 

In [157]:
# Constants
pace_threshold = 1
#euclidean_distance_threshold = 200

In [158]:
# Read in Split Timings
df = pd.read_csv('../../data/grade_adjusted_pace.csv', encoding = "ISO-8859-1")

In [159]:
len(df)

19579

In [160]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'team', 'nationality'], axis = 1)

In [161]:
df.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Male,Senior Men (23-39),1141.900000,4.022310,NaN,NaN,7.909121,NaN,4.586031,4.544534,5.117775,10.060820,5.096128,7.036173,4.797941,9.595926,11.377695,NaN,NaN,4.963401,6.456256,8.204950,5.493334,8.630844,5.997667,NaN,NaN,6.612290,NaN,NaN,10.976384,NaN,4.534051,NaN,13.508974,NaN,NaN,6.067253,NaN,NaN,6.668578,9.238605,7.145710
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),1156.983333,3.919652,NaN,NaN,8.170776,NaN,4.635109,4.563283,5.221164,10.041015,5.029801,7.042091,4.795822,9.629858,11.574286,NaN,NaN,5.201188,6.645315,8.614387,5.652586,8.717315,6.095252,NaN,NaN,7.230409,NaN,NaN,10.376782,NaN,4.599921,NaN,12.617061,NaN,NaN,6.130417,NaN,NaN,6.878126,8.948532,7.180423
2,2017,0d928f7482046900e19dba217d7befa5,Male,Senior Men (23-39),1193.000000,4.222026,NaN,NaN,7.905156,NaN,4.633745,4.893558,5.567362,11.493365,5.472986,7.746300,5.096753,10.315281,12.680109,NaN,NaN,5.318496,6.734412,8.245489,NaN,17.972368,6.169065,NaN,NaN,7.106542,NaN,NaN,9.575274,NaN,4.859741,NaN,13.073039,NaN,NaN,6.169017,NaN,NaN,7.251440,8.420044,7.866155
3,2017,93f20063908ba5d48724dcf31740a52a,Male,Senior Men (23-39),1203.650000,4.023243,NaN,NaN,7.917050,NaN,4.584668,4.548860,6.048279,11.598990,5.654883,8.373579,5.198476,10.403504,13.130630,NaN,NaN,5.751268,6.988664,9.161654,6.128629,8.382241,6.136537,NaN,NaN,7.006963,NaN,NaN,11.006976,NaN,4.797531,NaN,12.852566,NaN,NaN,5.909343,NaN,NaN,6.832343,8.483622,7.538354
4,2017,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),1211.633333,3.918719,NaN,NaN,7.675217,NaN,4.703272,4.636838,5.111509,10.047617,5.123262,6.787629,4.255418,9.460199,12.139484,NaN,NaN,5.066442,6.814816,8.156304,5.500183,9.514465,7.895553,NaN,NaN,9.910542,NaN,NaN,8.987909,NaN,4.310825,NaN,11.199020,NaN,NaN,6.555019,NaN,NaN,7.438097,9.071714,7.261669


In [162]:
df.shape

(19579, 45)

In [163]:
df.groupby('category').count()

,Year,name_mask,gender,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),7881,7881,7881,4931,5994,560,954,5990,801,7749,7435,7379,7389,7116,7098,6886,6834,6869,1697,816,5475,6215,6176,5779,5764,5339,1816,540,2726,1811,534,2722,4254,714,4262,714,3707,524,712,3705,523,711,4408,7877
Masters Men 2 (50-59),3393,3393,3393,1805,2592,220,376,2588,373,3312,3145,3116,3119,2966,2944,2848,2814,2838,675,330,2172,2462,2438,2230,2220,2004,736,183,974,733,183,970,1608,239,1606,239,1403,173,235,1398,172,236,1633,3388
Masters Men 3 (60-69),520,520,520,183,371,43,61,370,74,502,458,453,454,423,417,388,387,384,70,35,281,309,304,261,254,216,100,23,76,100,23,77,180,13,178,13,148,22,13,148,21,13,162,520
Masters Men 4 (70-79),29,29,29,4,23,1,4,23,2,28,27,25,26,23,23,21,21,20,2,1,14,15,15,12,12,7,3,0,2,3,0,2,5,0,4,0,4,0,0,4,0,0,4,29
Masters Women 1 (40-49),719,719,719,397,560,51,69,560,81,702,668,660,665,633,625,607,601,602,166,72,466,537,530,474,473,431,140,29,240,140,29,239,341,61,343,61,309,27,61,310,27,61,370,717
Masters Women 2 (50-59),287,287,287,133,220,15,37,220,26,278,261,259,258,239,237,227,228,224,51,23,171,191,187,169,168,150,60,12,69,60,12,69,117,20,116,20,100,12,20,101,12,20,121,285
Masters Women 3 (60-69),26,26,26,8,21,0,3,21,2,24,21,22,21,18,18,17,16,16,8,5,7,12,12,9,9,8,3,1,4,3,1,4,8,0,8,0,7,1,0,7,1,0,7,26
Senior Men (23-39),6042,6042,6042,3972,4555,501,700,4555,674,5943,5737,5703,5706,5511,5515,5366,5315,5374,1325,639,4360,4955,4917,4701,4687,4354,1493,367,2247,1481,355,2248,3418,586,3423,586,3046,344,580,3044,344,580,3628,6036
Senior Women (23-39),646,646,646,380,519,31,69,518,51,630,614,606,607,585,579,561,558,563,156,70,440,498,493,460,457,421,134,33,237,134,33,234,326,66,324,66,284,32,66,285,32,66,348,645


In [164]:
df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
2426,2017,440fde6abce3adaaf2a6dcd91a38b1a1,Female,Masters Women 2 (50-59),NaN,8.562574,NaN,NaN,17.653792,NaN,10.364921,9.877979,12.870413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.865936
2471,2017,6edbc123d2c45fa61b34a1f47eae4a2c,Female,Masters Women 2 (50-59),NaN,7.818773,NaN,NaN,17.388172,NaN,9.639663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.615536
2595,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),1753.933333,6.053996,NaN,NaN,10.565317,NaN,7.383456,6.948766,7.107237,14.893183,6.571403,25.937382,6.743396,13.864552,18.012636,12.221814,NaN,6.878377,8.603271,10.941285,7.356410,11.095256,9.064059,NaN,NaN,9.48551,NaN,NaN,15.406095,8.661238,NaN,15.031206,NaN,10.013872,NaN,NaN,21.334744,NaN,NaN,14.992368,11.406673
7405,2015,cbf15f23ef5ead61ed9491f0e92aa9b8,Female,Masters Women 2 (50-59),NaN,6.923786,NaN,NaN,13.935117,NaN,8.328200,9.976052,9.458563,20.022617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.440722
7546,2015,ea16550cfbbfd6cd4327356dc80126e6,Female,Masters Women 2 (50-59),NaN,6.853792,NaN,NaN,16.385161,NaN,10.039100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.092684
7559,2015,7c2c1575d30df2b22d3662dc49bc41bb,Female,Masters Women 2 (50-59),NaN,7.743180,NaN,NaN,15.976820,NaN,9.867329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.195776
7571,2015,7b3fd7188a39638af4c973ad75c306fc,Female,Masters Women 2 (50-59),NaN,7.902766,NaN,NaN,16.706283,NaN,10.382644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.663897
7575,2015,139eea5f34dcf73b372d65df0c63e41f,Female,Masters Women 2 (50-59),NaN,7.382012,NaN,NaN,17.289060,NaN,11.196514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.955862
7586,2015,a4c30f4dc95d2279739b89433807187b,Female,Masters Women 2 (50-59),NaN,7.587327,NaN,NaN,15.635876,NaN,12.445267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.889490
12320,2013,4e83e10fffd16e1331d47148f86ceba4,Female,Masters Women 2 (50-59),NaN,6.342371,NaN,NaN,12.143176,NaN,8.813523,8.915997,9.847056,22.174734,13.388014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.660696


In [165]:
df.loc[1613]

Year                                                        2017
name_mask                       4db319a671ab6e2b099df28df89f85c4
gender                                                      Male
category                                   Masters Men 1 (40-49)
time                                                     2721.83
Chamonix to Delevret                                     6.61208
Chamonix to La Charme                                        NaN
Chamonix to St-Gervais                                       NaN
Delevret to St-Gervais                                   14.6765
La Charme to St-Gervais                                      NaN
St-Gervais to Contamines                                 9.08209
Contamines to La Balme                                   10.3164
La Balme to Bonhomme                                     11.7457
Bonhomme to Chapieux                                     21.9041
Chapieux to Col Seigne                                    11.162
Col Seigne to Lac Combal 

In [166]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Male,Senior Men (23-39),2586.933333,6.035331,NaN,NaN,11.952882,NaN,8.690829,7.343943,8.266452,21.435356,9.665660,16.977948,11.202261,20.467689,27.006666,NaN,NaN,15.467233,12.143238,15.165375,10.851389,17.631890,14.193479,16.968304,NaN,NaN,26.090769,NaN,NaN,14.240774,NaN,33.262276,NaN,16.234132,NaN,NaN,26.841212,NaN,NaN,31.959612,16.670613
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Male,Masters Men 1 (40-49),2646.166667,6.442229,NaN,NaN,13.836006,NaN,9.983206,12.015395,9.547854,25.488731,10.384203,41.832387,8.818124,20.196235,27.514526,44.976277,NaN,8.447769,13.803480,16.020733,10.873650,18.337166,18.588515,NaN,NaN,16.094157,NaN,NaN,26.247875,14.690155,NaN,21.150690,NaN,11.521201,NaN,NaN,21.477477,NaN,NaN,19.999096,17.931485
5634,2015,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),2323.750000,6.070795,NaN,NaN,12.036136,NaN,7.961481,9.078972,8.896187,20.854417,8.802403,37.257986,9.436940,18.818602,23.771109,24.217995,10.215059,NaN,12.517009,16.272071,10.801730,18.053435,13.277690,NaN,NaN,14.565254,NaN,NaN,22.240332,10.601447,NaN,22.944244,NaN,10.362885,NaN,NaN,23.828806,NaN,NaN,19.029539,15.676501
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),2021.466667,5.884145,NaN,NaN,11.072769,NaN,7.237586,7.152124,7.751071,15.969241,7.015593,12.302947,7.684335,15.513640,20.789481,NaN,NaN,9.861807,10.000569,13.584382,8.442062,15.545788,11.862721,NaN,NaN,12.997492,NaN,NaN,19.841925,9.491116,NaN,24.590034,NaN,10.997682,NaN,NaN,27.539850,NaN,NaN,22.983264,13.171318
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Male,Senior Men (23-39),2193.000000,NaN,NaN,9.253519,NaN,NaN,6.880410,7.365577,7.860727,17.923312,7.881864,12.663928,7.277443,14.502471,16.824899,NaN,NaN,11.201339,10.200493,13.912742,10.561996,14.429777,14.286059,14.020444,NaN,NaN,21.128906,NaN,NaN,9.892935,NaN,24.742894,NaN,12.158528,NaN,NaN,24.587542,NaN,NaN,29.853607,13.887453


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [167]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [168]:
df.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,7.266289,NaN,NaN,15.532799,NaN,8.930764,8.691006,9.167193,18.398626,9.187301,14.948169,9.119055,15.798667,21.420210,NaN,NaN,11.692765,11.017576,17.415250,9.830807,14.513545,13.131314,NaN,NaN,15.131155,NaN,NaN,28.328126,11.713418,NaN,25.012946,NaN,12.904605,NaN,NaN,27.652533,NaN,NaN,14.058574,14.619279
10507,2013,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.333333,8.144478,NaN,NaN,19.271296,NaN,9.642389,10.366902,9.704504,20.920432,9.087811,16.486777,9.070313,20.216594,26.703589,NaN,NaN,12.111270,11.397867,20.561022,11.943891,25.271025,18.363321,NaN,27.459783,NaN,NaN,11.847977,NaN,11.259117,NaN,21.441124,NaN,NaN,10.665904,NaN,NaN,13.744685,NaN,NaN,15.464438
12954,2011,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),2370.466667,6.291042,NaN,NaN,13.352340,NaN,7.601578,8.232370,8.163062,17.507411,8.207470,14.001334,8.144209,16.986282,23.140380,NaN,NaN,14.227571,10.611208,16.118025,10.116775,21.082607,17.483814,NaN,28.103322,NaN,NaN,13.869012,NaN,10.539124,NaN,23.041056,NaN,NaN,11.376090,NaN,NaN,12.665705,NaN,NaN,13.950512
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),2707.316667,NaN,3.95927,NaN,NaN,16.919709,7.616574,8.812155,8.383940,19.903788,9.774195,20.540418,10.693645,16.402654,17.873384,NaN,NaN,11.659475,18.908073,17.658480,12.358289,16.853655,15.246886,21.854135,NaN,NaN,34.258426,NaN,NaN,12.958235,NaN,31.973159,NaN,17.822391,NaN,NaN,22.626849,NaN,NaN,33.354343,17.017172


# Merge

In [169]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [170]:
pb.head()

,name_mask,gender,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
10507,003cc0338235bb1862eb7b9bdd31ec9c,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
12954,006db776bb4048de7f357c632a8778ac,Male,2370.466667,6.291042,NaN,NaN,13.352340,NaN,7.601578,8.232370,8.163062,17.507411,8.207470,14.001334,8.144209,16.986282,23.140380,NaN,NaN,14.227571,10.611208,16.118025,10.116775,21.082607,17.483814,NaN,28.103322,NaN,NaN,13.869012,NaN,10.539124,NaN,23.041056,NaN,NaN,11.37609,NaN,NaN,12.665705,NaN,NaN,13.950512
8431,00a7ade381ebb6d25e1aacb0bc7129e9,Male,2456.833333,6.447828,NaN,NaN,13.348376,NaN,7.372549,8.561203,8.410570,21.468364,8.831547,16.356587,8.941040,19.300434,21.240002,NaN,NaN,13.622008,11.228366,19.377304,10.981530,19.428857,14.851546,NaN,NaN,17.096019,NaN,NaN,27.569446,13.717590,NaN,30.006364,NaN,13.442033,NaN,NaN,26.923847,NaN,NaN,19.454713,15.749089
15237,00b39183bebaffe426e376f7efc8ae1b,Male,2192.983333,NaN,NaN,7.954464,NaN,NaN,7.887864,8.875614,8.701940,22.531220,8.445644,15.605037,8.763024,17.176300,21.035220,NaN,NaN,10.163004,10.578611,16.320717,9.142428,15.640365,12.071651,13.441064,NaN,NaN,25.771483,NaN,NaN,10.724453,NaN,24.691941,NaN,11.136782,NaN,NaN,22.266261,NaN,NaN,21.568665,14.369294
6180,011b597a8c4121461b659f50dbd19d5a,Male,2595.100000,6.405832,NaN,NaN,13.950975,NaN,8.853058,8.911670,10.114929,22.874503,9.435525,40.921057,9.710321,20.834153,24.631194,19.855748,12.013688,NaN,15.059527,20.913705,11.524357,17.918324,15.093004,NaN,NaN,17.465190,NaN,NaN,23.457891,13.140284,NaN,23.168439,NaN,12.826204,NaN,NaN,26.668431,NaN,NaN,21.985892,17.109356


In [171]:
npb.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,7.266289,NaN,NaN,15.532799,NaN,8.930764,8.691006,9.167193,18.398626,9.187301,14.948169,9.119055,15.798667,21.420210,NaN,NaN,11.692765,11.017576,17.415250,9.830807,14.513545,13.131314,NaN,NaN,15.131155,NaN,NaN,28.328126,11.713418,NaN,25.012946,NaN,12.904605,NaN,NaN,27.652533,NaN,NaN,14.058574,14.619279
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.333333,8.144478,NaN,NaN,19.271296,NaN,9.642389,10.366902,9.704504,20.920432,9.087811,16.486777,9.070313,20.216594,26.703589,NaN,NaN,12.111270,11.397867,20.561022,11.943891,25.271025,18.363321,NaN,27.459783,NaN,NaN,11.847977,NaN,11.259117,NaN,21.441124,NaN,NaN,10.665904,NaN,NaN,13.744685,NaN,NaN,15.464438
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),2707.316667,NaN,3.95927,NaN,NaN,16.919709,7.616574,8.812155,8.383940,19.903788,9.774195,20.540418,10.693645,16.402654,17.873384,NaN,NaN,11.659475,18.908073,17.658480,12.358289,16.853655,15.246886,21.854135,NaN,NaN,34.258426,NaN,NaN,12.958235,NaN,31.973159,NaN,17.822391,NaN,NaN,22.626849,NaN,NaN,33.354343,17.017172
3763,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.916667,6.787531,NaN,NaN,12.991573,NaN,8.780805,9.784232,10.028771,22.801885,10.286722,45.359350,8.309509,19.001834,22.182000,32.491224,NaN,10.354818,13.996885,21.789332,10.014032,18.991100,14.817767,NaN,NaN,18.186531,NaN,NaN,27.526617,13.294451,NaN,24.528890,NaN,15.154217,NaN,NaN,31.175771,NaN,NaN,28.152909,18.271550
17546,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),2255.450000,NaN,NaN,NaN,NaN,58.309199,8.543596,9.436650,9.098266,22.676455,9.074746,19.451557,9.042763,19.442948,21.485740,NaN,NaN,10.375427,12.388797,16.734207,9.539701,17.002277,12.145464,13.851458,NaN,NaN,24.856775,NaN,NaN,10.548965,NaN,24.595129,NaN,11.280939,NaN,NaN,17.563603,NaN,NaN,16.446703,16.690929


In [172]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [173]:
#merge = merge.groupby('name_mask').head(1)

In [174]:
merge.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,7.266289,NaN,NaN,15.532799,NaN,8.930764,8.691006,9.167193,18.398626,9.187301,14.948169,9.119055,15.798667,21.420210,NaN,NaN,11.692765,11.017576,17.415250,9.830807,14.513545,13.131314,NaN,NaN,15.131155,NaN,NaN,28.328126,11.713418,NaN,25.012946,NaN,12.904605,NaN,NaN,27.652533,NaN,NaN,14.058574,14.619279,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.333333,8.144478,NaN,NaN,19.271296,NaN,9.642389,10.366902,9.704504,20.920432,9.087811,16.486777,9.070313,20.216594,26.703589,NaN,NaN,12.111270,11.397867,20.561022,11.943891,25.271025,18.363321,NaN,27.459783,NaN,NaN,11.847977,NaN,11.259117,NaN,21.441124,NaN,NaN,10.665904,NaN,NaN,13.744685,NaN,NaN,15.464438,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),2707.316667,NaN,3.95927,NaN,NaN,16.919709,7.616574,8.812155,8.383940,19.903788,9.774195,20.540418,10.693645,16.402654,17.873384,NaN,NaN,11.659475,18.908073,17.658480,12.358289,16.853655,15.246886,21.854135,NaN,NaN,34.258426,NaN,NaN,12.958235,NaN,31.973159,NaN,17.822391,NaN,NaN,22.626849,NaN,NaN,33.354343,17.017172,Male,2370.466667,6.291042,NaN,NaN,13.352340,NaN,7.601578,8.232370,8.163062,17.507411,8.207470,14.001334,8.144209,16.986282,23.140380,NaN,NaN,14.227571,10.611208,16.118025,10.116775,21.082607,17.483814,NaN,28.103322,NaN,NaN,13.869012,NaN,10.539124,NaN,23.041056,NaN,NaN,11.37609,NaN

In [175]:
first_df = len(merge)

In [176]:
merge.loc[1613]

Year                                                           2015
name_mask                          fbac5ac0b66892696a502c9f3de3ee0e
gender                                                         Male
category                                          Youth Men (20-22)
time                                                        1938.88
Chamonix to Delevret                                        5.70309
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      10.8349
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                     6.5764
Contamines to La Balme                                      6.63003
La Balme to Bonhomme                                        7.40487
Bonhomme to Chapieux                                        16.6954
Chapieux to Col Seigne                          

In [177]:
merge.groupby('category').count()

,Year,name_mask,gender,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),690,690,690,690,484,65,96,484,88,687,688,689,690,689,688,687,674,689,144,77,612,687,686,689,690,690,289,122,279,289,122,279,632,57,633,57,510,122,57,510,122,57,568,690,690,690,509,56,113,509,60,685,689,686,689,688,690,687,677,690,106,59,631,685,682,689,690,690,308,92,290,308,92,288,610,77,613,77,521,92,76,521,92,77,598,690
Masters Men 2 (50-59),290,290,290,290,233,14,30,233,22,290,289,290,290,289,290,288,288,288,79,39,250,289,290,290,290,290,99,42,148,99,42,149,255,34,256,34,213,42,34,213,42,34,248,290,290,290,167,33,67,168,51,286,289,286,288,290,290,290,283,290,32,24,265,290,283,290,290,290,164,39,87,164,39,86,268,22,268,22,228,39,22,229,39,22,251,290
Masters Men 3 (60-69),31,31,31,31,26,5,0,25,4,31,31,31,31,31,31,31,31,31,7,4,26,31,31,31,31,31,8,6,17,8,6,17,26,5,26,5,20,6,5,20,6,5,25,31,31,31,12,0,15,12,4,31,31,30,31,31,31,31,31,31,0,0,31,31,31,31,31,31,23,6,2,23,6,2,30,1,30,1,24,6,1,24,6,1,25,31
Masters Men 4 (70-79),1,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1
Masters Women 1 (40-49),46,46,46,46,35,3,6,35,4,46,46,46,46,46,46,45,46,46,16,9,36,46,46,46,46,46,16,3,27,16,3,27,43,3,43,3,40,3,3,40,3,3,43,46,46,46,38,1,7,38,1,46,46,46,46,46,46,46,46,46,10,3,43,46,46,46,46,46,13,6,27,13,6,27,36,10,36,10,30,6,10,30,6,10,40,46
Masters Women 2 (50-59),14,14,14,14,11,1,2,11,0,14,14,14,14,14,14,14,14,14,4,1,13,14,14,14,14,14,5,2,7,5,2,7,12,2,12,2,10,2,2,10,2,2,12,14,14,14,12,0,1,12,1,14,14,14,14,14,14,14,14,14,0,0,14,14,14,14,14,14,7,2,5,7,2,5,13,1,13,1,11,2,1,11,2,1,12,14
Masters Women 3 (60-69),2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,2,2,0,0,2,0,0,2,2,0,2,0,2,0,0,2,0,0,2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,0,0,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,0,2,0,0,2,0,0,2,2
Senior Men (23-39),501,501,501,501,318,59,89,319,84,497,499,500,500,499,499,501,485,499,114,57,443,499,498,501,500,500,247,62,191,247,61,

In [178]:
#df = df[~df["AVG_Pace"] > 12 & df.category == 'Masters Men 3 (60-69)']

merge[(merge["AVG_Pace"] <12) & (merge.category == 'Masters Women 2 (50-59)')]



#df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
347,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),1753.933333,6.053996,NaN,NaN,10.565317,NaN,7.383456,6.948766,7.107237,14.893183,6.571403,25.937382,6.743396,13.864552,18.012636,12.221814,NaN,6.878377,8.603271,10.941285,7.35641,11.095256,9.064059,NaN,NaN,9.48551,NaN,NaN,15.406095,8.661238,NaN,15.031206,NaN,10.013872,NaN,NaN,21.334744,NaN,NaN,14.992368,11.406673,Female,1667.866667,5.746024,NaN,NaN,11.056911,NaN,6.831332,6.402153,7.293652,15.645764,6.822641,11.001048,6.59505,13.15877,16.833091,NaN,NaN,7.694778,8.831446,11.216946,7.467715,11.406009,9.275492,NaN,NaN,9.424791,NaN,NaN,13.582816,8.142974,NaN,13.390512,NaN,9.183071,NaN,NaN,17.308187,NaN,NaN,16.192393,10.437648


##### Runners with no similar runners

In [179]:
#merge = merge.drop([30])
#merge = merge.drop([347])

In [180]:
merge = merge[merge.name_mask != '0bf54733836ed02a86125cec3615d37c']
# merge = merge[merge.name_mask != '4d5f9076cc4be3cdf23a51be2c1e7d8c']
# merge = merge[merge.name_mask != '738138c60e7b32635a37f6d0757c1512']
# merge = merge[merge.name_mask != '972253ab3ee99e5c6bcee7a8eba23ebf']
# merge = merge[merge.name_mask != 'fbac5ac0b66892696a502c9f3de3ee0e']
# merge = merge[merge.name_mask != '1e4ba9a22b3fecb9136ecfcf7f470868']
# merge = merge[merge.name_mask != '32038d973773361675b8797c7a45f2bd']
# merge = merge[merge.name_mask != 'c10477f93ccf5878840d9fe8b3d1291e']
# merge = merge[merge.name_mask != '2ae85673eb8e4b8fc082e85b200d5064']
merge = merge[merge.name_mask != '5453919ab2a372d03ff30b1cf2494dae']
# merge = merge[merge.name_mask != '3afce9f734b439fd08ff984ecbef2828']
# merge = merge[merge.name_mask != 'f7f8b87a909450888d40bbb33ee52de8']
# merge = merge[merge.name_mask != '3c3a924c0b32b8eda3e90af5dfd69ce7']
# merge = merge[merge.name_mask != '0bf54733836ed02a86125cec3615d37c']

In [181]:
merge = merge.reset_index(drop=True)

In [182]:
merge.loc[537]

Year                                                           2009
name_mask                          545b0e38a62911b316d40a57a91422f3
gender                                                         Male
category                                      Masters Men 1 (40-49)
time                                                        2344.65
Chamonix to Delevret                                            NaN
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                      9.25352
Delevret to St-Gervais                                          NaN
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    7.87968
Contamines to La Balme                                      8.69389
La Balme to Bonhomme                                        9.55725
Bonhomme to Chapieux                                        21.7192
Chapieux to Col Seigne                          

In [183]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [184]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [185]:
merge.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,7.266289,NaN,NaN,15.532799,NaN,8.930764,8.691006,9.167193,18.398626,9.187301,14.948169,9.119055,15.798667,21.420210,NaN,NaN,11.692765,11.017576,17.415250,9.830807,14.513545,13.131314,NaN,NaN,15.131155,NaN,NaN,28.328126,11.713418,NaN,25.012946,NaN,12.904605,NaN,NaN,27.652533,NaN,NaN,14.058574,14.619279,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.333333,8.144478,NaN,NaN,19.271296,NaN,9.642389,10.366902,9.704504,20.920432,9.087811,16.486777,9.070313,20.216594,26.703589,NaN,NaN,12.111270,11.397867,20.561022,11.943891,25.271025,18.363321,NaN,27.459783,NaN,NaN,11.847977,NaN,11.259117,NaN,21.441124,NaN,NaN,10.665904,NaN,NaN,13.744685,NaN,NaN,15.464438,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),2707.316667,NaN,3.95927,NaN,NaN,16.919709,7.616574,8.812155,8.383940,19.903788,9.774195,20.540418,10.693645,16.402654,17.873384,NaN,NaN,11.659475,18.908073,17.658480,12.358289,16.853655,15.246886,21.854135,NaN,NaN,34.258426,NaN,NaN,12.958235,NaN,31.973159,NaN,17.822391,NaN,NaN,22.626849,NaN,NaN,33.354343,17.017172,Male,2370.466667,6.291042,NaN,NaN,13.352340,NaN,7.601578,8.232370,8.163062,17.507411,8.207470,14.001334,8.144209,16.986282,23.140380,NaN,NaN,14.227571,10.611208,16.118025,10.116775,21.082607,17.483814,NaN,28.103322,NaN,NaN,13.869012,NaN,10.539124,NaN,23.041056,NaN,NaN,11.37609,NaN

In [186]:
merge[merge.category == 'Masters Women 2 (50-59)']

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
112,2015,1320ec5e4c7a443d78f71103526fc704,Female,Masters Women 2 (50-59),2622.816667,8.106214,NaN,NaN,16.567527,NaN,9.695557,9.050126,10.577048,22.742471,9.660635,41.773209,12.575523,20.494835,37.712676,7.170131,12.197516,NaN,14.329368,17.013921,12.476443,17.885898,16.421648,NaN,NaN,15.860996,NaN,NaN,24.320583,10.752334,NaN,19.352041,NaN,12.087714,NaN,NaN,25.519059,NaN,NaN,20.591161,16.997385,Female,2494.933333,6.834194,NaN,NaN,14.030265,NaN,8.365008,9.003974,9.980209,20.214063,10.074677,15.782568,9.841714,19.178279,25.278305,NaN,NaN,13.211429,12.280142,18.424654,11.262362,17.213049,13.211383,NaN,NaN,15.036433,NaN,NaN,31.901263,NaN,10.848421,NaN,41.153266,NaN,NaN,13.875024,NaN,NaN,17.116075,25.132978,16.218739
113,2013,13213e63d3265f0750495f08ab12d901,Female,Masters Women 2 (50-59),2358.233333,7.246691,NaN,NaN,15.794455,NaN,9.202054,8.343423,9.518090,22.425595,8.597392,18.309436,8.417590,18.106033,24.016847,NaN,NaN,10.847830,13.240649,18.594915,9.854780,18.580365,14.138431,13.568474,NaN,NaN,27.480090,NaN,NaN,12.699103,NaN,29.303209,NaN,10.792827,NaN,NaN,20.252983,NaN,NaN,15.298335,15.192900,Female,2264.133333,7.317618,NaN,NaN,16.967938,NaN,8.141432,8.405440,8.351043,20.167851,7.925077,14.628612,7.703408,17.481687,22.411356,NaN,NaN,9.803153,10.441707,14.232995,8.515695,19.917956,13.808147,NaN,22.439860,NaN,NaN,13.52351,NaN,11.401804,NaN,24.549272,NaN,NaN,12.505575,NaN,NaN,15.244512,NaN,NaN,13.734159
303,2016,30f977102cb8714bf2b596874ef040d2,Female,Masters Women 2 (50-59),2670.966667,7.289620,NaN,NaN,15.243393,NaN,9.635573,11.526472,10.772861,20.616759,11.249469,36.944347,10.700003,18.472497,26.400511,37.555442,NaN,9.478178,12.162795,16.122079,11.010641,17.369777,16.689379,NaN,NaN,18.399047,NaN,NaN,29.600750,13.402696,NaN,20.804208,NaN,12.794590,NaN,NaN,25.181008,NaN,NaN,18.477209,17.515972,Female,2044.333333,6.973248,NaN,NaN,13.582279,NaN,7.540231,8.914555,8.894621,14.998808,8.585333,11.995225,6.773066,19.327579,16.996916,NaN,NaN,9.815835,10.080973,12.741185,8.842760,15.926799,11.893998,NaN,18.594476,NaN,NaN,12.81416,NaN,1

In [187]:
merge.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,7.266289,NaN,NaN,15.532799,NaN,8.930764,8.691006,9.167193,18.398626,9.187301,14.948169,9.119055,15.798667,21.420210,NaN,NaN,11.692765,11.017576,17.415250,9.830807,14.513545,13.131314,NaN,NaN,15.131155,NaN,NaN,28.328126,11.713418,NaN,25.012946,NaN,12.904605,NaN,NaN,27.652533,NaN,NaN,14.058574,14.619279,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.333333,8.144478,NaN,NaN,19.271296,NaN,9.642389,10.366902,9.704504,20.920432,9.087811,16.486777,9.070313,20.216594,26.703589,NaN,NaN,12.111270,11.397867,20.561022,11.943891,25.271025,18.363321,NaN,27.459783,NaN,NaN,11.847977,NaN,11.259117,NaN,21.441124,NaN,NaN,10.665904,NaN,NaN,13.744685,NaN,NaN,15.464438,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),2707.316667,NaN,3.95927,NaN,NaN,16.919709,7.616574,8.812155,8.383940,19.903788,9.774195,20.540418,10.693645,16.402654,17.873384,NaN,NaN,11.659475,18.908073,17.658480,12.358289,16.853655,15.246886,21.854135,NaN,NaN,34.258426,NaN,NaN,12.958235,NaN,31.973159,NaN,17.822391,NaN,NaN,22.626849,NaN,NaN,33.354343,17.017172,Male,2370.466667,6.291042,NaN,NaN,13.352340,NaN,7.601578,8.232370,8.163062,17.507411,8.207470,14.001334,8.144209,16.986282,23.140380,NaN,NaN,14.227571,10.611208,16.118025,10.116775,21.082607,17.483814,NaN,28.103322,NaN,NaN,13.869012,NaN,10.539124,NaN,23.041056,NaN,NaN,11.37609,NaN

In [188]:
pb.head()

,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
1,Male,2183.316667,7.139367,NaN,NaN,12.333471,NaN,8.380004,7.847289,8.700374,16.629400,8.583323,13.166935,8.722759,15.642581,20.298004,NaN,NaN,10.476882,10.254821,13.831665,10.947283,12.524722,12.697190,10.550873,NaN,NaN,20.464448,NaN,NaN,10.224230,NaN,27.759326,NaN,12.979213,NaN,NaN,27.953023,NaN,NaN,26.694601,13.950074
2,Male,2370.466667,6.291042,NaN,NaN,13.352340,NaN,7.601578,8.232370,8.163062,17.507411,8.207470,14.001334,8.144209,16.986282,23.140380,NaN,NaN,14.227571,10.611208,16.118025,10.116775,21.082607,17.483814,NaN,28.103322,NaN,NaN,13.869012,NaN,10.539124,NaN,23.041056,NaN,NaN,11.37609,NaN,NaN,12.665705,NaN,NaN,13.950512
3,Male,2456.833333,6.447828,NaN,NaN,13.348376,NaN,7.372549,8.561203,8.410570,21.468364,8.831547,16.356587,8.941040,19.300434,21.240002,NaN,NaN,13.622008,11.228366,19.377304,10.981530,19.428857,14.851546,NaN,NaN,17.096019,NaN,NaN,27.569446,13.717590,NaN,30.006364,NaN,13.442033,NaN,NaN,26.923847,NaN,NaN,19.454713,15.749089
4,Male,2192.983333,NaN,NaN,7.954464,NaN,NaN,7.887864,8.875614,8.701940,22.531220,8.445644,15.605037,8.763024,17.176300,21.035220,NaN,NaN,10.163004,10.578611,16.320717,9.142428,15.640365,12.071651,13.441064,NaN,NaN,25.771483,NaN,NaN,10.724453,NaN,24.691941,NaN,11.136782,NaN,NaN,22.266261,NaN,NaN,21.568665,14.369294


In [189]:
pb.to_csv('../../data/GAP_pb_times.csv')

In [190]:
npb.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,7.266289,NaN,NaN,15.532799,NaN,8.930764,8.691006,9.167193,18.398626,9.187301,14.948169,9.119055,15.798667,21.420210,NaN,NaN,11.692765,11.017576,17.415250,9.830807,14.513545,13.131314,NaN,NaN,15.131155,NaN,NaN,28.328126,11.713418,NaN,25.012946,NaN,12.904605,NaN,NaN,27.652533,NaN,NaN,14.058574,14.619279
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.333333,8.144478,NaN,NaN,19.271296,NaN,9.642389,10.366902,9.704504,20.920432,9.087811,16.486777,9.070313,20.216594,26.703589,NaN,NaN,12.111270,11.397867,20.561022,11.943891,25.271025,18.363321,NaN,27.459783,NaN,NaN,11.847977,NaN,11.259117,NaN,21.441124,NaN,NaN,10.665904,NaN,NaN,13.744685,NaN,NaN,15.464438
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),2707.316667,NaN,3.95927,NaN,NaN,16.919709,7.616574,8.812155,8.383940,19.903788,9.774195,20.540418,10.693645,16.402654,17.873384,NaN,NaN,11.659475,18.908073,17.658480,12.358289,16.853655,15.246886,21.854135,NaN,NaN,34.258426,NaN,NaN,12.958235,NaN,31.973159,NaN,17.822391,NaN,NaN,22.626849,NaN,NaN,33.354343,17.017172
3,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.916667,6.787531,NaN,NaN,12.991573,NaN,8.780805,9.784232,10.028771,22.801885,10.286722,45.359350,8.309509,19.001834,22.182000,32.491224,NaN,10.354818,13.996885,21.789332,10.014032,18.991100,14.817767,NaN,NaN,18.186531,NaN,NaN,27.526617,13.294451,NaN,24.528890,NaN,15.154217,NaN,NaN,31.175771,NaN,NaN,28.152909,18.271550
4,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),2255.450000,NaN,NaN,NaN,NaN,58.309199,8.543596,9.436650,9.098266,22.676455,9.074746,19.451557,9.042763,19.442948,21.485740,NaN,NaN,10.375427,12.388797,16.734207,9.539701,17.002277,12.145464,13.851458,NaN,NaN,24.856775,NaN,NaN,10.548965,NaN,24.595129,NaN,11.280939,NaN,NaN,17.563603,NaN,NaN,16.446703,16.690929


In [191]:
merge.iloc[67]

Year                                                           2015
name_mask                          0c57fa08817368009f3f6d89823435cc
gender                                                         Male
category                                      Masters Men 1 (40-49)
time                                                        2691.73
Chamonix to Delevret                                         6.8062
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      13.6417
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    8.86396
Contamines to La Balme                                      10.4866
La Balme to Bonhomme                                        9.62461
Bonhomme to Chapieux                                        20.5243
Chapieux to Col Seigne                          

# Define Functions

In [192]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_gender = npb[npb.gender == query.gender]
    similar_times = same_gender[(same_gender.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_gender.AVG_Pace < query.AVG_Pace + pace_threshold)]
    #find all runners either 0.5 minutes/kilometre faster or slower than the query runner
   # if len(similar_times) = 1:
    #    similar_times
    return similar_times
    #else:
     #   return None

# Show sample output
find_similar_times(67)

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
3,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.916667,6.787531,NaN,NaN,12.991573,NaN,8.780805,9.784232,10.028771,22.801885,10.286722,45.359350,8.309509,19.001834,22.182000,32.491224,NaN,10.354818,13.996885,21.789332,10.014032,18.991100,14.817767,NaN,NaN,18.186531,NaN,NaN,27.526617,13.294451,NaN,24.528890,NaN,15.154217,NaN,NaN,31.175771,NaN,NaN,28.152909,18.271550
9,2013,0148b715a2efb7898f74afd345fceb1f,Male,Senior Men (23-39),2731.616667,6.020399,NaN,NaN,12.206608,NaN,10.949763,8.168911,9.015242,22.867901,9.689779,22.244722,10.422383,25.971435,29.693408,NaN,NaN,14.517671,14.292426,23.706793,11.950741,20.834004,16.389120,17.052797,NaN,NaN,31.695516,NaN,NaN,16.203944,NaN,27.499464,NaN,14.618053,NaN,NaN,23.370560,NaN,NaN,25.033639,17.683970
10,2016,015d1c46b9e76fa644f6a2ece59f33c0,Male,Masters Men 1 (40-49),2725.733333,6.448761,NaN,NaN,13.439559,NaN,8.607670,8.928977,9.809460,20.940237,10.411337,33.825707,9.392436,19.802625,25.540426,49.250778,NaN,9.685845,15.187739,17.034191,12.661381,22.698526,18.078076,NaN,NaN,17.437259,NaN,NaN,30.622520,14.025924,NaN,21.711176,NaN,14.403082,NaN,NaN,28.599075,NaN,NaN,23.495858,18.481545
19,2016,02e896e2f857d562615509710d1b1e43,Male,Masters Men 1 (40-49),2759.916667,7.186963,NaN,NaN,16.206760,NaN,9.668292,11.150044,11.211483,25.323692,9.572199,40.767196,10.572849,20.040148,27.825795,18.464341,NaN,12.398199,13.188495,16.819338,12.026086,20.790769,17.892916,NaN,NaN,17.706851,NaN,NaN,31.295543,13.364974,NaN,21.298455,NaN,14.265247,NaN,NaN,25.594181,NaN,NaN,19.840152,17.778839
32,2016,0549f8f7450a83ba7086af5ae73da3b9,Male,Masters Men 2 (50-59),2733.366667,7.339082,NaN,NaN,16.706283,NaN,9.386095,9.022724,9.843923,24.485290,9.440550,41.814633,9.975225,22.408591,31.143265,45.189375,NaN,9.400501,14.879160,18.732745,10.928446,20.066578,16.241493,NaN,NaN,16.453613,NaN,NaN,36.538999,12.064394,NaN,27.713468,NaN,13.431917,NaN,NaN,26.931359,NaN,NaN,24.648200,19.391436
38,2008,06eb94c7166267fc273d39c697fc4f9d,Male,Masters Men 2 (50-59),2543.716667,NaN,NaN,NaN,NaN,65.709491,8.340469,8.751580,8.794364,24.208023,8.338114,18.889373,8.803290,18.967902,23.083041,NaN,NaN,14.267202,12.701722,17.455789,9.738338,19.569372,15.156809,18.609879,NaN,NaN,34.111727,NaN,NaN,13.591304,NaN,26.373398,NaN,12.295098,NaN,NaN,24.211930,NaN,NaN,22.526301,18.891066
41,2015,07328403d0f0e7c8d5b3c2948cf7b6f5,Male,Masters Men 1 (40-49),2638.700000,6.445962,NaN,NaN,14.153163,NaN,9.098446,8.502071,8.833527,18.510853,8.968221,35.831815,8.816005,17.563123,28.685880,57.599218,9.640147,NaN,11.549984,15.996410,11.159618,18.961376,15.331960,NaN,NaN,18.379617,NaN,NaN,28.358718,13.456818,NaN,30.490419,NaN,14.466309,NaN,NaN,29.635763,NaN,NaN,28.872130,18.772302
47,2008,08dc28972fa0c34f5653f04547a0bf8b,Male,Masters Men 2 (50-59),2476.600000,NaN,NaN,NaN,NaN,63.018980,8.722184,8.936189,9.950446,23.996773,9.576219,15.782568,9.907410,19.246143,20.363534,NaN,NaN,14.934590,13.251514,18.971921,13.618604,21.366338,15.796110,14.913653,NaN,NaN,25.025047,NaN,NaN,10.762174,NaN,21.471696,NaN,10.757420,NaN,NaN,22.416506,NaN,NaN,18.23879

In [193]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(194)

449     0.998004
1227    0.997490
476     0.996885
192     0.996124
193     0.995660
1088    0.994786
59      0.993737
1086    0.992707
309     0.992174
1594    0.991700
dtype: float64

In [194]:
def euc_distance(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    
    return euclidean(a,b)

def euc_sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: euc_distance(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = True) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

euc_sims(194)

192     3.407565
193     3.991351
1086    4.147050
1227    4.534453
476     4.559133
928     5.190838
813     5.198578
1594    5.349451
449     5.474815
1088    5.655259
dtype: float64

In [195]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
   # if weighter.sum() > 3:
    if len(weighter) > 1:
        weighter = weighter / weighter.sum()
        return weighter
    
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
    #else:
     #   return np.nan
    
    #elif weight0 <= 1:
     #   return 0
    

weight(194)

449     0.100309
1227    0.100258
476     0.100197
192     0.100120
193     0.100074
1088    0.099986
59      0.099880
1086    0.099777
309     0.099723
1594    0.099676
dtype: float64

In [196]:
def euc_weight(index):
    weighter = euc_sims(index)
    if len(weighter) > 1:
        weighter = 1 -(weighter / weighter.sum())
        weighter = weighter / weighter.sum()
        return weighter
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
euc_weight(194)

192     0.103142
193     0.101776
1086    0.101412
1227    0.100506
476     0.100448
928     0.098971
813     0.098953
1594    0.098600
449     0.098307
1088    0.097885
dtype: float64

In [197]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    else: 
        nearest_neighbours['weights'] = weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        
        return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(194)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb,weights,weighted_pace
449,2008,4658955cfea9cf728e90a9cc02278081,Male,Masters Men 1 (40-49),1390.783333,NaN,3.139357,NaN,NaN,12.964380,5.255395,5.554109,5.960555,14.034976,5.779498,10.515794,6.232661,12.846597,15.669928,NaN,NaN,6.605715,8.216461,9.595414,6.142328,9.717131,7.755432,7.597648,NaN,NaN,12.322678,NaN,NaN,6.448776,NaN,10.720562,NaN,7.124404,NaN,NaN,11.982013,NaN,NaN,10.116769,8.845774,Male,1368.600000,NaN,NaN,6.07206,NaN,NaN,5.174963,5.096916,5.905727,13.902944,5.552378,10.024623,5.923254,11.238227,15.317703,NaN,NaN,9.733402,7.260300,8.719787,6.073833,9.400973,6.966002,6.957917,NaN,NaN,12.098316,NaN,NaN,6.558661,NaN,11.500147,NaN,7.006802,NaN,NaN,9.991271,NaN,NaN,10.303528,8.555641,0.100309,0.858210
1227,2017,bf5f4fb86e38559cba1d9b622aa26fd4,Male,Senior Men (23-39),1340.366667,4.288286,NaN,NaN,8.983492,NaN,5.078171,4.912308,5.788239,14.589510,5.945315,10.450699,5.952923,14.495685,16.243318,NaN,NaN,6.060391,7.881804,9.149493,6.510491,9.684704,6.770834,NaN,NaN,7.305700,NaN,NaN,12.328547,NaN,4.969524,NaN,15.924154,NaN,NaN,6.677838,NaN,NaN,8.595017,10.732676,8.721630,Male,1319.700000,4.488935,NaN,NaN,9.253077,NaN,5.053632,4.745007,6.012250,13.691694,5.478011,24.718331,5.713450,12.140814,15.350468,2.9207,6.661055,NaN,7.275512,9.939989,5.561829,9.536083,7.382611,NaN,NaN,6.386416,NaN,NaN,10.676583,5.632019,NaN,10.144790,NaN,6.186117,NaN,NaN,10.930300,NaN,NaN,7.784270,8.546558,0.100258,0.856858
476,2009,4bcd7473cdbc13d04aa01409261a202d,Male,Senior Men (23-39),1356.750000,NaN,NaN,5.726463,NaN,NaN,5.155877,5.447383,5.861865,13.751108,5.858889,9.687313,5.592653,12.249396,15.989388,NaN,NaN,6.325126,8.259922,9.534606,5.763891,10.146781,7.870531,7.416592,NaN,NaN,11.878268,NaN,NaN,6.578342,NaN,10.720562,NaN,6.870232,NaN,NaN,11.305912,NaN,NaN,8.972374,8.563629,Male,1255.683333,4.522532,NaN,NaN,8.797162,NaN,4.716905,4.848849,5.326120,11.631998,5.235817,8.894338,4.838207,10.098117,12.041189,NaN,NaN,5.301058,6.643142,8.164412,5.447099,10.427811,7.256252,NaN,9.82903,NaN,NaN,7.938909,NaN,5.841948,NaN,10.002121,NaN,NaN,6.734981,NaN,Na

In [198]:
def euc_find_nearest_neighbours(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1: 
        nearest_neighbours['weights'] = euc_weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    else: 
        nearest_neighbours['weights'] = euc_weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb'] 
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
# Show sample output    
euc_find_nearest_neighbours(194)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb,weights,weighted_pace
192,2011,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),1236.716667,4.330283,NaN,NaN,8.868523,NaN,4.741444,4.858944,5.399746,12.410986,5.225767,8.977186,4.831849,10.091331,12.016615,NaN,NaN,5.113999,6.373679,8.549526,5.515595,10.417002,7.376355,NaN,9.848051,NaN,NaN,7.379378,NaN,5.956754,NaN,9.248013,NaN,NaN,6.292097,NaN,NaN,6.879058,NaN,NaN,7.421834,Male,1156.983333,3.919652,NaN,NaN,8.170776,NaN,4.635109,4.563283,5.221164,10.041015,5.029801,7.042091,4.795822,9.629858,11.574286,NaN,NaN,5.201188,6.645315,8.614387,5.652586,8.717315,6.095252,NaN,NaN,7.230409,NaN,NaN,10.376782,NaN,4.599921,NaN,12.617061,NaN,NaN,6.130417,NaN,NaN,6.878126,8.948532,7.180423,0.103142,0.740600
193,2009,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Youth Men (20-22),1293.300000,NaN,NaN,5.599008,NaN,NaN,4.771435,4.486844,5.089578,12.338368,5.327268,9.947693,5.414638,11.971155,13.335412,NaN,NaN,5.881258,8.281653,10.410234,5.948828,10.346745,6.830885,7.399157,NaN,NaN,12.495265,NaN,NaN,6.607864,NaN,10.129504,NaN,6.243022,NaN,NaN,9.623172,NaN,NaN,10.279686,8.206899,Male,1156.983333,3.919652,NaN,NaN,8.170776,NaN,4.635109,4.563283,5.221164,10.041015,5.029801,7.042091,4.795822,9.629858,11.574286,NaN,NaN,5.201188,6.645315,8.614387,5.652586,8.717315,6.095252,NaN,NaN,7.230409,NaN,NaN,10.376782,NaN,4.599921,NaN,12.617061,NaN,NaN,6.130417,NaN,NaN,6.878126,8.948532,7.180423,0.101776,0.730797
1086,2017,a82e067a0e3f059af79e29cad5b6b70f,Male,Senior Men (23-39),1362.216667,4.163231,NaN,NaN,8.051842,NaN,5.038636,5.125761,5.883796,11.262309,5.990538,8.343990,5.893584,11.340023,13.212543,NaN,NaN,5.790899,8.607617,10.746701,6.688579,9.760366,7.517727,NaN,NaN,8.567440,NaN,NaN,13.864262,NaN,5.959404,NaN,17.933464,NaN,NaN,7.498970,NaN,NaN,7.758583,10.335316,8.555649,Male,1328.183333,4.267755,NaN,NaN,7.944801,NaN,5.164056,4.768083,5.755343,11.731022,5.001662,7.793642,4.973838,9.819876,11.918320,NaN,NaN,5.648227,6.890874,8.784648,5.685121,10.095440,7.928081,7.664706,NaN,NaN,13.086374,NaN,NaN,7.441022,NaN,13.680945,NaN,7.472152,NaN,NaN,12.177331,NaN,NaN

####  means

In [199]:
def get_mean_pb(neighbours):
    mean = neighbours.AVG_Pace_pb.mean() 
    return mean
    
# Show sample output    
neighbours = find_nearest_neighbours(10)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.336569322399999

In [200]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(10)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.336054222418666

In [201]:
def euc_get_weight_mean(neighbours):
    ew_mean = neighbours.weighted_pace.sum()
    return ew_mean

neighbours = euc_find_nearest_neighbours(10)
euc_get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.283324187749983

In [202]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_npb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


9.6834414795932382

## Get Best

In [203]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(5)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


14.346461740122836

In [204]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


13.036878109238266

# Calculate Predictions

In [205]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
    neighbours1 = euc_find_nearest_neighbours(index)
    row['euc_weighted_mean'] = euc_get_weight_mean(neighbours1)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [206]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,   'euc_weighted_mean':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean', 'euc_weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,14.619279,13.950074,13.036878,13.565867,13.565562,13.506101
1,15.464438,13.950074,12.861522,15.536794,15.536859,15.486665
2,17.017172,13.950512,15.311925,16.276751,16.277098,16.063747
3,18.271550,15.749089,15.196247,16.285705,16.285688,16.505808
4,16.690929,14.369294,13.480787,14.541288,14.541413,15.033237
5,16.706899,17.109356,15.228655,15.987593,15.987507,16.080788
6,14.785444,13.073505,10.778007,13.879241,13.878426,13.514202
7,17.351609,18.480502,14.814869,16.355725,16.355268,16.595877
8,15.267995,12.107432,13.500007,15.014275,15.014326,14.943728
9,17.683970,15.162386,13.394402,16.214946,16.215023,16.462069


In [207]:
df.iloc[[66]]

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
66,12.280343,11.108782,10.059073,11.338787,11.3389,11.516979


Comb dataframe contains the information of the gender and category of each runner. This will be used in a later notebook to test how accurate the predictions are in comparison to male, female, senior men, etc...

In [208]:
comb.join(df)

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,Senior Men (23-39),Male,14.619279,13.950074,13.036878,13.565867,13.565562,13.506101
1,Senior Men (23-39),Male,15.464438,13.950074,12.861522,15.536794,15.536859,15.486665
2,Senior Men (23-39),Male,17.017172,13.950512,15.311925,16.276751,16.277098,16.063747
3,Masters Men 1 (40-49),Male,18.271550,15.749089,15.196247,16.285705,16.285688,16.505808
4,Masters Men 1 (40-49),Male,16.690929,14.369294,13.480787,14.541288,14.541413,15.033237
5,Masters Men 1 (40-49),Male,16.706899,17.109356,15.228655,15.987593,15.987507,16.080788
6,Masters Men 1 (40-49),Male,14.785444,13.073505,10.778007,13.879241,13.878426,13.514202
7,Senior Men (23-39),Male,17.351609,18.480502,14.814869,16.355725,16.355268,16.595877
8,Senior Men (23-39),Male,15.267995,12.107432,13.500007,15.014275,15.014326,14.943728
9,Senior Men (23-39),Male,17.683970,15.162386,13.394402,16.214946,16.215023,16.462069


In [209]:
df = comb.join(df)

In [210]:
df.sort_values(by = 'weighted_mean')

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
192,Senior Men (23-39),Male,7.421834,7.180423,6.977774,7.682719,7.679069,7.659356
813,Senior Men (23-39),Male,7.600457,7.145710,7.353236,7.857635,7.855667,7.752244
928,Masters Men 1 (40-49),Male,7.783725,7.481216,7.494138,8.074543,8.074404,7.794377
194,Youth Men (20-22),Male,7.938553,7.180423,7.680335,8.153168,8.152715,7.862870
1227,Senior Men (23-39),Male,8.721630,8.546558,7.888716,8.164185,8.164209,8.328115
449,Masters Men 1 (40-49),Male,8.845774,8.555641,8.001004,8.169531,8.169441,8.424479
193,Youth Men (20-22),Male,8.206899,7.180423,7.939952,8.193093,8.193211,8.103981
946,Senior Men (23-39),Male,7.984021,7.538354,7.506325,8.194653,8.197354,8.116264
943,Senior Men (23-39),Male,9.129139,9.159386,7.987314,8.229008,8.229131,8.440435
1633,Senior Men (23-39),Male,8.599302,8.572921,8.084793,8.257345,8.258522,8.275042


In [211]:
df.to_csv('../../data/GAP_pace_predictions.csv')

In [215]:
df.query_pb.mean()

13.988889581614865

# Coverage

In [212]:
predict = pd.read_csv('../../data/GAP_pace_predictions.csv')

In [213]:
predict = len(predict)

In [214]:
(predict / first_df) * 100

99.87775061124694